In [29]:
import os
import pandas as pd
import numpy as np
import pandas as pd
#import seaborn as sns
from matplotlib import pyplot as plt

#fft packages
from scipy import fftpack
import math
from scipy import fft
from scipy import ndimage
from scipy import signal

In [ ]:
if file_name.endswith('.FLX1'):
            print(file_name)
    #        print(i)
            x = pd.read_table(file_name,delimiter=r"\s+")
        

In [12]:
folder = '/home/sreenath/Documents/Work/NOAA/SPLASH/Tower_Data/raw_data/Kettle_Ponds/2023/ascii_files/'
file = '2023-06-30T000000_smart3-00565-floor.data'
fname = folder + file
header_list = ['DATAH',	'Seconds',	'Nanoseconds',	'Sequence Number',	'Diagnostic Value',
 'Diagnostic Value 2',	'DS Diagnostic Value',	'Date',	'Time',	'CO2 Absorptance',
 'H2O Absorptance',	'CO2 (mmol/m^3)',	'CO2 (mg/m^3)',	'H2O (mmol/m^3)',	'H2O (g/m^3)',
 'Temperature (C)',	'Pressure (kPa)',	'None1',	'None2',	'None3',	'None4',	'Cooler Voltage (V)',
 'Chopper Cooler Voltage (V)',	'Vin SmartFlux (V)',	'CO2 (umol/mol)',	'H2O (mmol/mol)',
 'Dew Point (C)',	'CO2 Signal Strength',	'H2O Sample',	'H2O Reference',	'CO2 Sample',
 'CO2 Reference',	'HIT Power (W)',	'Vin HIT (V)',	'Vin DSI (V)',	'U (m/s)',	'V (m/s)',	'W (m/s)',	'T (C)',
 'Anemometer Diagnostics',	'CHK',
]
df = pd.read_table(fname,delimiter=r"\s+",names=header_list,skiprows=8)

In [19]:
def belltaper(var, n): #-- following Stull 1999
    W = [] # weight function 
    for i in np.arange(n):
        if (i/n > 0.1) & (i/n < 0.9):
            W.append(1)
        else:
            W.append(math.sin(np.pi*i/n)**2)
    out = var * W
    return(out)


In [23]:
w_data = df['W (m/s)'].values
q_data = df['H2O (g/m^3)'].values

In [27]:
q_data.size

18000

In [30]:
N = w_data.size
data_w = w_data-w_data.mean()

data_var = q_data-q_data.mean()


# #-- apply bell taper to space series
data_bt_w = belltaper(data_w,N)

data_bt_var = belltaper(data_var,N)

#-- calculate wave vectors
# time interval
sampling_frequency = 10
dt = 1/sampling_frequency
# total duration
L = dt*N
# nyquist aliasing frequency
nyquist = np.power(dt,-1)/2
# wave vector increments
delta_f = np.power(L,-1)
# frequencies according to Frederik's python scripts
f = np.linspace(delta_f,nyquist,int(N/2)) # !!! might be an issue when nx is uneven
# calculate spectral contributions
S = (np.abs(fftpack.fft(data_bt_w)*np.conj(fftpack.fft(data_bt_var)))[:int(N/2)])           
#mean spectral power in x and y directions
S_norm = (S/np.sum(S))